<a href="https://colab.research.google.com/github/ChibaU-ST/lecture-ai-engineering/blob/master/training_notebook_balanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 ひらがなGPT - Transformerで日本語文章生成

このノートブックでは、Transformerアーキテクチャを使って日本語（ひらがな）の文章を生成するモデルを学習します。

## 📚 学習内容
- **Self-Attention（自己注意機構）**: 文脈を理解する仕組み
- **Transformer**: 現代のAIの基礎となるアーキテクチャ
- **言語モデル**: 次の文字を予測することで文章を生成

## 📋 必要なもの
1. `input.txt` - 前処理済みのテキストファイル（01_前処理.ipynb で作成）
2. GPU（推奨）- 学習を高速化

---

## 1️⃣ ライブラリのインポートと設定

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
from datetime import datetime

print("="*70)
print("  ひらがなGPT - Transformer言語モデル")
print("="*70)
print(f"実行開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"PyTorchバージョン: {torch.__version__}")
print(f"CUDA利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
print("="*70)

  ひらがなGPT - Transformer言語モデル
実行開始時刻: 2025-10-06 13:13:00
PyTorchバージョン: 2.8.0+cu126
CUDA利用可能: True
GPU名: NVIDIA A100-SXM4-40GB


## 2️⃣ ハイパーパラメータの設定

これらのパラメータがモデルの性能を決定します。

### 過学習対策のポイント
- **適度なモデルサイズ**: データ量に合わせた大きさ
- **Dropout**: ランダムにニューロンを無効化
- **Weight Decay**: パラメータが大きくなりすぎるのを防ぐ
- **Gradient Clipping**: 学習の安定化
- **Early Stopping**: 過学習の兆候で自動停止

In [18]:
# === データ処理 ===
batch_size = 32      # 一度に処理する文章数
block_size = 128     # 一度に見る文字数（64→128に拡大）

# === 学習制御 ===
max_iters = 8000     # 最大学習回数
eval_interval = 100  # 評価間隔
learning_rate = 3e-4 # 学習率
eval_iters = 200     # 評価時のバッチ数

# === Transformer構造（データ量に最適化）===
n_embd = 256         # 埋め込み次元数（128→256）
n_head = 8           # Attention Headの数（4→8）
n_layer = 6          # Transformerブロックの層数（4→6）
dropout = 0.25       # Dropout率（0.2→0.25で過学習対策強化）

# === 過学習対策 ===
weight_decay = 0.1   # L2正則化
grad_clip = 1.0      # 勾配クリッピング

# === チェックポイント ===
checkpoint_interval = 100  # 保存間隔
checkpoint_dir = 'checkpoints'

# === Early Stopping ===
patience = 10        # 改善なしで待つ回数
min_delta = 0.005    # 改善とみなす最小差

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("\n📊 ハイパーパラメータ設定")
print("-"*70)
print(f"  デバイス: {device}")
print(f"  バッチサイズ: {batch_size}")
print(f"  コンテキスト長: {block_size}文字")
print(f"\n  【モデル構造】")
print(f"  埋め込み次元: {n_embd}")
print(f"  Attention Head数: {n_head}")
print(f"  Transformer層数: {n_layer}")
print(f"  予想パラメータ数: 約{(n_embd * n_embd * n_layer * 12) / 1e6:.1f}M")
print(f"\n  【過学習対策】")
print(f"  Dropout率: {dropout}")
print(f"  Weight Decay: {weight_decay}")
print(f"  Gradient Clipping: {grad_clip}")
print(f"  Early Stopping: {patience}回")
print(f"\n  【学習設定】")
print(f"  学習率: {learning_rate}")
print(f"  最大イテレーション: {max_iters}")
print("-"*70)

os.makedirs(checkpoint_dir, exist_ok=True)


📊 ハイパーパラメータ設定
----------------------------------------------------------------------
  デバイス: cuda
  バッチサイズ: 32
  コンテキスト長: 128文字

  【モデル構造】
  埋め込み次元: 256
  Attention Head数: 8
  Transformer層数: 6
  予想パラメータ数: 約4.7M

  【過学習対策】
  Dropout率: 0.25
  Weight Decay: 0.1
  Gradient Clipping: 1.0
  Early Stopping: 10回

  【学習設定】
  学習率: 0.0003
  最大イテレーション: 8000
----------------------------------------------------------------------


## 3️⃣ データの読み込みと前処理

`input.txt`（前処理済み）をアップロードしてください。

In [13]:
print("\n📖 データ読み込み中...")
print("-"*70)

if not os.path.exists('input.txt'):
    print("❌ input.txt が見つかりません")
    print("\n前処理ノートブック（01_前処理.ipynb）を先に実行してください。")
    print("または、前処理済みの input.txt をアップロードしてください。")
    raise FileNotFoundError("input.txt")

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("✅ input.txt を読み込みました")

# 文字セットの作成
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"\n📊 データ統計")
print(f"  総文字数: {len(text):,} 文字")
print(f"  語彙サイズ: {vocab_size} 種類")
print(f"  文字セット: {''.join(chars)}")

if vocab_size > 70:
    print(f"\n  ⚠️ 語彙サイズが大きい ({vocab_size})")
    print(f"     前処理で濁点分離を有効にすると改善する可能性があります")

# エンコード・デコード関数
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# データの分割
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print(f"\n📂 データ分割")
print(f"  訓練データ: {len(train_data):,} 文字 (90%)")
print(f"  検証データ: {len(val_data):,} 文字 (10%)")
print("-"*70)

print("\n📝 サンプルテキスト（最初の200文字）:")
print("-"*70)
print(text[:200])
print("-"*70)


📖 データ読み込み中...
----------------------------------------------------------------------
✅ input.txt を読み込みました

📊 データ統計
  総文字数: 944,046 文字
  語彙サイズ: 85 種類
  文字セット: 
 AK―…○、。「」ぁあぃいうぇえぉおかきくけこさしすせそたちっつてとなにぬねのはひふへほまみむめもゃやゅゆょよらりるれろわをん゛゜イシハャリンヴー新！（）？ＡＢＨＫＮ

  ⚠️ 語彙サイズが大きい (85)
     前処理で濁点分離を有効にすると改善する可能性があります

📂 データ分割
  訓練データ: 849,641 文字 (90%)
  検証データ: 94,405 文字 (10%)
----------------------------------------------------------------------

📝 サンプルテキスト（最初の200文字）:
----------------------------------------------------------------------
なつめ そうせき 「こころ」
し゛ょう せんせい と わたし
いち
わたくしは そのひとを つねに せんせいと よんて゛いた。 た゛から ここて゛も たた゛ せんせいと かくた゛けて゛ ほんみょうは うちあけない。 これは せけんを はは゛かる えんりょと いうよりも、 そのほうか゛ わたくしに とって しせ゛んた゛からて゛ある。 わたくしは そのひとの きおくを よひ゛おこすこ゛とに、 すく゛ 
----------------------------------------------------------------------


In [14]:
# データ取得関数
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

print("✅ データ処理関数を定義しました")

✅ データ処理関数を定義しました


## 4️⃣ Transformerモデルの定義

### 🧠 Transformerの構成要素

1. **Self-Attention**: 文章内の単語間の関係を学習
2. **Multi-Head Attention**: 複数の視点から文脈を捉える
3. **Feed Forward**: 各単語の特徴を変換
4. **Layer Normalization**: 学習を安定化
5. **Residual Connection**: 深いネットワークの学習を可能に

In [19]:
print("\n🏗️  モデル構築中...")
print("-"*70)

class Head(nn.Module):
    """Self-Attention Head"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """複数のAttention Headを並列実行"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """Feed Forward Network"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Transformerブロック"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    """GPT言語モデル"""

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

print("✅ すべてのクラスを定義しました")


🏗️  モデル構築中...
----------------------------------------------------------------------
✅ すべてのクラスを定義しました


In [20]:
# モデルのインスタンス化
model = GPTLanguageModel()
m = model.to(device)

# パラメータ数の計算
n_params = sum(p.numel() for p in m.parameters())

print("\n✨ モデル構築完了")
print("-"*70)
print(f"  総パラメータ数: {n_params:,} ({n_params/1e6:.2f}M)")
print(f"  デバイス: {device}")
print(f"  モデルサイズ: 約{n_params * 4 / 1024 / 1024:.1f}MB (float32)")
print("-"*70)

# モデル構造の表示
print("\n🏗️  モデル構造:")
print(f"  Token Embedding: {vocab_size} → {n_embd}")
print(f"  Position Embedding: {block_size} → {n_embd}")
print(f"  Transformer Blocks: {n_layer}層")
print(f"    ├─ Multi-Head Attention: {n_head}heads × {n_embd//n_head}dim")
print(f"    └─ Feed Forward: {n_embd} → {4*n_embd} → {n_embd}")
print(f"  Output Layer: {n_embd} → {vocab_size}")
print("-"*70)


✨ モデル構築完了
----------------------------------------------------------------------
  総パラメータ数: 4,810,837 (4.81M)
  デバイス: cuda
  モデルサイズ: 約18.4MB (float32)
----------------------------------------------------------------------

🏗️  モデル構造:
  Token Embedding: 85 → 256
  Position Embedding: 128 → 256
  Transformer Blocks: 6層
    ├─ Multi-Head Attention: 8heads × 32dim
    └─ Feed Forward: 256 → 1024 → 256
  Output Layer: 256 → 85
----------------------------------------------------------------------


## 5️⃣ モデルの学習

AdamWオプティマイザ（Weight Decay付き）を使用して学習します。

In [21]:
# Weight Decay付きオプティマイザ
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

checkpoints = []
best_val_loss = float('inf')
patience_counter = 0

print("\n🚀 学習開始")
print("="*70)
print(f"開始時刻: {datetime.now().strftime('%H:%M:%S')}")
print(f"最大イテレーション: {max_iters}")
print(f"評価間隔: {eval_interval}回")
print(f"チェックポイント保存: {checkpoint_interval}回ごと")
print(f"\n過学習対策:")
print(f"  - Dropout: {dropout}")
print(f"  - Weight Decay: {weight_decay}")
print(f"  - Gradient Clipping: {grad_clip}")
print(f"  - Early Stopping: patience={patience}, min_delta={min_delta}")
print("="*70)

start_time = datetime.now()

for iter in range(max_iters):
    # 定期的な評価
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()

        # Train/Val Gapの計算（過学習の指標）
        gap = losses['val'].item() - losses['train'].item()

        # プログレスバー風の表示
        progress = iter / max_iters * 100
        bar_length = 30
        filled = int(bar_length * iter / max_iters)
        bar = '█' * filled + '░' * (bar_length - filled)

        elapsed = (datetime.now() - start_time).total_seconds()
        eta = elapsed / (iter + 1) * (max_iters - iter) if iter > 0 else 0

        print(f"\n[{bar}] {progress:5.1f}%")
        print(f"ステップ {iter:5d}/{max_iters} | "
              f"{int(elapsed//60)}:{int(elapsed%60):02d} (残り{int(eta//60)}分)")
        print(f"Train: {losses['train']:.4f} | Val: {losses['val']:.4f} | Gap: {gap:.4f}")

        # Early Stoppingチェック
        if losses['val'] < best_val_loss - min_delta:
            best_val_loss = losses['val']
            patience_counter = 0
            print(f"  ✨ 新記録！ Val Loss: {best_val_loss:.4f}")
        else:
            patience_counter += 1
            if patience_counter > 0:
                print(f"  ⏳ 改善なし: {patience_counter}/{patience}回")

        # 過学習警告
        if gap > 0.5:
            print(f"  ⚠️ 過学習の兆候 (Gap={gap:.4f})")
        elif gap < 0.2:
            print(f"  ✅ 良好な汎化 (Gap={gap:.4f})")

        if patience_counter >= patience:
            print(f"\n🛑 Early Stopping: {patience}回連続で改善なし")
            break

    # チェックポイント保存
    if iter % checkpoint_interval == 0 and iter > 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'model_iter_{iter}.pt')
        torch.save({
            'iter': iter,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': losses['train'].item() if iter % eval_interval == 0 else None,
            'val_loss': losses['val'].item() if iter % eval_interval == 0 else None,
        }, checkpoint_path)
        checkpoints.append((iter, checkpoint_path))
        print(f"  💾 チェックポイント保存: iter_{iter}.pt")

    # 学習ステップ
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()

    # Gradient Clipping（勾配爆発防止）
    torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

    optimizer.step()

# 最終モデル保存
final_path = os.path.join(checkpoint_dir, 'model_final.pt')
torch.save({
    'iter': iter,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, final_path)
checkpoints.append((iter, final_path))

total_time = (datetime.now() - start_time).total_seconds()
final_gap = losses['val'].item() - losses['train'].item()

print("\n")
print("="*70)
print("✅ 学習完了！")
print(f"終了時刻: {datetime.now().strftime('%H:%M:%S')}")
print(f"所要時間: {int(total_time//60)}分{int(total_time%60)}秒")
print(f"\n最終Loss:")
print(f"  Train: {losses['train']:.4f}")
print(f"  Val: {losses['val']:.4f}")
print(f"  Gap: {final_gap:.4f}")
print(f"\nベストVal Loss: {best_val_loss:.4f}")
print(f"保存されたチェックポイント: {len(checkpoints)}個")

# 最終評価
if final_gap < 0.3:
    print("\n🎉 優秀！過学習を抑えて学習できました")
elif final_gap < 0.5:
    print("\n👍 良好！適度な汎化性能です")
else:
    print("\n⚠️ 過学習気味です。データ量を増やすか、モデルを小さくすることを検討してください")

print("="*70)


🚀 学習開始
開始時刻: 13:14:00
最大イテレーション: 8000
評価間隔: 100回
チェックポイント保存: 100回ごと

過学習対策:
  - Dropout: 0.25
  - Weight Decay: 0.1
  - Gradient Clipping: 1.0
  - Early Stopping: patience=10, min_delta=0.005

[░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]   0.0%
ステップ     0/8000 | 0:06 (残り0分)
Train: 4.6146 | Val: 4.6225 | Gap: 0.0079
  ✨ 新記録！ Val Loss: 4.6225
  ✅ 良好な汎化 (Gap=0.0079)

[░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]   1.2%
ステップ   100/8000 | 0:19 (残り26分)
Train: 2.8258 | Val: 2.8485 | Gap: 0.0228
  ✨ 新記録！ Val Loss: 2.8485
  ✅ 良好な汎化 (Gap=0.0228)
  💾 チェックポイント保存: iter_100.pt

[░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]   2.5%
ステップ   200/8000 | 0:33 (残り21分)
Train: 2.7801 | Val: 2.8036 | Gap: 0.0235
  ✨ 新記録！ Val Loss: 2.8036
  ✅ 良好な汎化 (Gap=0.0235)
  💾 チェックポイント保存: iter_200.pt

[█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]   3.8%
ステップ   300/8000 | 0:46 (残り19分)
Train: 2.7490 | Val: 2.7691 | Gap: 0.0201
  ✨ 新記録！ Val Loss: 2.7691
  ✅ 良好な汎化 (Gap=0.0201)
  💾 チェックポイント保存: iter_300.pt

[█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░]   5.0%
ステップ   400/8000 | 0:59 (残り18分)


## 6️⃣ 学習経過の確認 - 各チェックポイントでの文章生成

学習が進むにつれて、生成される文章がどのように改善されるかを確認します。

In [22]:
print("\n" + "="*70)
print("📊 学習経過の可視化 - 各段階での文章生成")
print("="*70)
print("各チェックポイントで300文字を生成し、学習の進捗を確認します。")
print("最初は意味不明な文字列ですが、徐々に文章らしくなっていきます。")
print("="*70)

for idx, (iter_num, checkpoint_path) in enumerate(checkpoints, 1):
    print(f"\n{'='*70}")
    print(f"📈 チェックポイント {idx}/{len(checkpoints)}: イテレーション {iter_num}回")
    print(f"{'-'*70}")

    # モデルロード
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Loss情報があれば表示
    if 'train_loss' in checkpoint and checkpoint['train_loss'] is not None:
        train_l = checkpoint['train_loss']
        val_l = checkpoint['val_loss']
        gap = val_l - train_l
        print(f"Train: {train_l:.4f} | Val: {val_l:.4f} | Gap: {gap:.4f}")
        print(f"{'-'*70}")

    # 文章生成
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    with torch.no_grad():
        generated = decode(model.generate(context, max_new_tokens=300)[0].tolist())

    print(generated)
    print(f"{'-'*70}")

    # 分析コメント
    if iter_num <= 500:
        print("💭 学習初期 - 文字の出現頻度を学習中")
    elif iter_num <= 2000:
        print("💭 学習中期 - 単語や文字列のパターンを獲得")
    elif iter_num <= 5000:
        print("💭 学習後期 - 文章構造や文脈を理解し始めています")
    else:
        print("💭 学習完了 - より自然で意味のある文章を生成")

print("\n" + "="*70)
print("🎉 すべてのチェックポイントのテスト完了")
print("="*70)


📊 学習経過の可視化 - 各段階での文章生成
各チェックポイントで300文字を生成し、学習の進捗を確認します。
最初は意味不明な文字列ですが、徐々に文章らしくなっていきます。

📈 チェックポイント 1/80: イテレーション 100回
----------------------------------------------------------------------
Train: 2.8258 | Val: 2.8485 | Gap: 0.0228
----------------------------------------------------------------------

「い し゛ちょうのか゛くにの な して゛ 。 たまして としＮたしけと いいいわの なとこんとあた゛ のきたし゛ゆうし゛、 ちに ろせんは そうに おこと ころとは ほうよくなったらないまくか゛ますし゛ふ゛を きまえしてかほとこと゛ついひとかていし …の あかいた゛んの しそれとか゛ねて゛えて゛よりちんかせた゛ら またりは おしのぁ に もははと゛ょうと こうて゛ くさなっき そこのを なら きいせいって゛にもはな うもと゛を おもった」ときいの いいますくのて゛んもんりかきまた。ぃんかえていに その て゛れするょうこにめて゛は へ まちのろせた゛つ Ｎこう すを し゛して゛さるしんなかんに 
----------------------------------------------------------------------
💭 学習初期 - 文字の出現頻度を学習中

📈 チェックポイント 2/80: イテレーション 200回
----------------------------------------------------------------------
Train: 2.7801 | Val: 2.8036 | Gap: 0.0235
----------------------------------------------------------------------

「か゛もようか けかるまいて゛ たし゛ょう あに た゛ とかくしら こと なる。 あった゛、 なか゛ むか゛きに うして゛するし゛くる きに いのを の いうはいにっていう

## 7️⃣ インタラクティブな文章生成

最終モデルを使って、自由に文章を生成してみましょう。

In [23]:
# 最終モデルをロード
checkpoint = torch.load(final_path, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("\n🎮 インタラクティブ生成モード")
print("="*70)
print("下のセルのプロンプトを変更して、文章を生成してみましょう。")
print("="*70)


🎮 インタラクティブ生成モード
下のセルのプロンプトを変更して、文章を生成してみましょう。


In [24]:
# プロンプトを設定（空の場合はランダム生成）
prompt = "わたくし は"  # ← ここを変更してみてください
max_generate = 500     # 生成する文字数

print(f"\n💬 プロンプト: '{prompt}'")
print(f"📝 生成文字数: {max_generate}")
print("-"*70)

if prompt:
    context = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
else:
    context = torch.zeros((1, 1), dtype=torch.long, device=device)

with torch.no_grad():
    generated = decode(model.generate(context, max_new_tokens=max_generate)[0].tolist())

print(generated)
print("-"*70)


💬 プロンプト: 'わたくし は'
📝 生成文字数: 500
----------------------------------------------------------------------
わたくし はなしているのて゛す。 わたしは しす゛かて゛は、 せんせいと ふとうを おいて しは゛らく にき゛った。 ひっきょうな もんた゛いは けいたろうの とおりなと゛を よるほと゛ いろいろ ひとりも なかった。 しかし わたしは それか゛ ゆうきゅうしなくなっていたような こたえた。 きくか゛ むねを ある せんせいの おくて゛す。 わたしは たた゛ わたくしの うちわを したものか゛、 この あたまと それから すく゛ おもい きっと はたして しす゛かしないのて゛、 むろん し゛ふ゛んには こ゛ろく しておくか゛、 かのし゛ょに うったえの えんりょなく みせました。 そのひも かれの ねつは その はし゛めから つい むき゛わらほ゛うを にし゛ょうしていたのた゛と いうことに、 ある おとこと いう ことは゛は ないと こたえました。 しかし もう て゛んわの いっている りょうけんの ほうか゛ ことさらに もってから かねていないと おもって、 せ゛んたいから、 いちにち とさ゛てとも しんは゜いしている やさしい ふ゛んちょうたを みうけて きたのを また ひっくり みたのて゛あ
----------------------------------------------------------------------


## 🎓 まとめ

このノートブックでは、以下のことを学びました：

### 📚 Transformerの基礎
1. **Self-Attention機構** - 文章内の単語間の関係を学習
2. **Multi-Head Attention** - 複数の視点から文脈を捉える
3. **Feed Forward Network** - 各単語の特徴を変換
4. **Layer Normalization** - 学習の安定化
5. **Residual Connection** - 深いネットワークの学習を可能に

### 🛡️ 過学習対策
1. **Dropout** - ランダムにニューロンを無効化
2. **Weight Decay** - L2正則化でパラメータの増大を抑制
3. **Gradient Clipping** - 勾配爆発の防止
4. **Early Stopping** - 過学習の兆候で自動停止
5. **適切なモデルサイズ** - データ量に応じた設計

### 💡 さらに学ぶには
- より大きなデータセットで学習
- ハイパーパラメータの調整
- Temperature パラメータによる生成制御
- Beam Searchなどの高度な生成手法

---

お疲れ様でした！🎉